<a href="https://colab.research.google.com/github/YeralyK/TTS_Learning/blob/main/XTTSfine_tuning_fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
set -e

if [ ! -d /content/py310 ]; then
  python3 -m venv /content/py310
fi

source /content/py310/bin/activate

python -m pip install -U pip wheel "setuptools<81"

python -m pip install -U "TTS==0.22.0" datasets soundfile num2words tqdm scipy

python -c "import setuptools, pkg_resources; print('setuptools:', setuptools.__version__, '| pkg_resources OK')"

  Using cached scipy-1.15.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
INFO: pip is looking at multiple versions of scipy to determine which version is compatible with other requirements. This could take a while.
  Using cached scipy-1.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.15.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached scipy-1.14.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached scipy-1.13.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached scipy-1.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
INFO: pip is still looking

<string>:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.


In [ ]:
!source /content/py310/bin/activate && \
  pip install -q --no-cache-dir torch==2.2.2+cu121 torchaudio==2.2.2+cu121 --index-url https://download.pytorch.org/whl/cu121

!source /content/py310/bin/activate && \
  pip install -q --no-cache-dir TTS==0.22.0 transformers==4.37.2 datasets pandas tqdm soundfile num2words huggingface_hub librosa


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requ

In [ ]:
%%bash
source /content/py310/bin/activate
python - <<'PY'
import sys, torch
print("Python:", sys.version)
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device:", torch.cuda.get_device_name(0))
import TTS
print("TTS:", TTS.__version__)
PY


Python: 3.10.12 (main, Jan 26 2026, 14:55:28) [GCC 11.4.0]
Torch: 2.2.2+cu121
CUDA available: True
CUDA device: Tesla T4
TTS: 0.22.0


In [ ]:
%%bash
source /content/py310/bin/activate
python - <<'PY'
try:
    from huggingface_hub import notebook_login
    notebook_login()
except Exception as e:
    print("HF login skipped / failed (ok for public datasets).", e)
PY


HF login skipped / failed (ok for public datasets). The `notebook_login` function can only be used in a notebook (Jupyter or Colab) and you need the `ipywidgets` module: `pip install ipywidgets`.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
%%bash
set -e
source /content/py310/bin/activate

python - <<'PY'
import os, re, io, subprocess
import numpy as np
import soundfile as sf
from tqdm import tqdm
from datasets import load_dataset, Audio
from num2words import num2words
from scipy.signal import resample_poly

HF_DATASET = "Kostya165/syntetic_necoarc_rus"
HF_SPLIT   = "train"

TARGET_SR  = 22050
DATASET_DIR = "/content/xtts_dataset"
WAVS_DIR    = os.path.join(DATASET_DIR, "wavs")
os.makedirs(WAVS_DIR, exist_ok=True)

def normalize_russian_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = text.lower()

    def replace_numbers(m):
        try:
            return num2words(int(m.group(0)), lang="ru")
        except Exception:
            return m.group(0)

    text = re.sub(r"\d+", replace_numbers, text)
    text = re.sub(r"[^а-яё \.,!?\-]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def build_hf_audio_index():
    roots = [
        os.path.expanduser("~/.cache/huggingface/datasets/downloads/extracted"),
        os.path.expanduser("~/.cache/huggingface/hub"),
        os.path.expanduser("~/.cache/huggingface/datasets"),
    ]
    exts = (".wav", ".flac", ".ogg", ".mp3", ".m4a")
    index = {}
    for root in roots:
        if not os.path.isdir(root):
            continue
        for dirpath, _, files in os.walk(root):
            for f in files:
                if f.lower().endswith(exts) and f not in index:
                    index[f] = os.path.join(dirpath, f)
    return index

def try_read_soundfile(path_or_bytes):
    try:
        if isinstance(path_or_bytes, (bytes, bytearray)):
            return sf.read(io.BytesIO(path_or_bytes), dtype="float32")
        return sf.read(path_or_bytes, dtype="float32")
    except Exception:
        return None, None

def ffmpeg_to_wav(in_path, out_path):
    cmd = ["ffmpeg", "-y", "-i", in_path, "-ac", "1", "-ar", str(TARGET_SR), out_path]
    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=False)

def read_audio_any(audio_dict, audio_index):
    if not isinstance(audio_dict, dict):
        return None, None

    b = audio_dict.get("bytes", None)
    p = audio_dict.get("path", None)

    if b is not None:
        arr, sr = try_read_soundfile(b)
        if arr is not None:
            return arr, int(sr)

    if p and os.path.isabs(p) and os.path.exists(p):
        arr, sr = try_read_soundfile(p)
        if arr is not None:
            return arr, int(sr)

        tmp = "/tmp/_tmp_conv.wav"
        ffmpeg_to_wav(p, tmp)
        arr, sr = try_read_soundfile(tmp)
        if arr is not None:
            return arr, int(sr)

    if p:
        base = os.path.basename(p)
        resolved = audio_index.get(base)
        if resolved and os.path.exists(resolved):
            arr, sr = try_read_soundfile(resolved)
            if arr is not None:
                return arr, int(sr)

            tmp = "/tmp/_tmp_conv.wav"
            ffmpeg_to_wav(resolved, tmp)
            arr, sr = try_read_soundfile(tmp)
            if arr is not None:
                return arr, int(sr)

    return None, None

print("Loading dataset:", HF_DATASET)
ds = load_dataset(HF_DATASET, split=HF_SPLIT)
ds = ds.cast_column("audio", Audio(decode=False))

print("Indexing HF cache for audio files...")
audio_index = build_hf_audio_index()
print("Indexed audio files:", len(audio_index))

text_keys = ["text", "sentence", "transcript", "normalized_text"]

lines = []
kept = 0
skipped_audio = 0
skipped_text = 0

for item in tqdm(ds, desc="Saving wavs"):

    raw_txt = ""
    for k in text_keys:
        if k in item and isinstance(item[k], str):
            raw_txt = item[k]
            break
    txt = normalize_russian_text(raw_txt)
    if not txt:
        skipped_text += 1
        continue

    arr, sr = read_audio_any(item.get("audio"), audio_index)
    if arr is None or sr is None:
        skipped_audio += 1
        continue

    arr = np.asarray(arr, dtype=np.float32)
    if arr.ndim == 2:
        arr = arr.mean(axis=1)
    arr = np.clip(arr, -1.0, 1.0)

    if sr != TARGET_SR:
        g = np.gcd(sr, TARGET_SR)
        up = TARGET_SR // g
        down = sr // g
        arr = resample_poly(arr, up, down).astype(np.float32)
        sr = TARGET_SR

    utt_id = f"necoarc_{kept:06d}"
    wav_path = os.path.join(WAVS_DIR, f"{utt_id}.wav")
    sf.write(wav_path, arr, sr, subtype="PCM_16")

    lines.append(f"{utt_id}|{txt}|{txt}")
    kept += 1

meta_path = os.path.join(DATASET_DIR, "metadata.csv")
with open(meta_path, "w", encoding="utf-8") as f:
    f.write("\n".join(lines) + ("\n" if lines else ""))

print("\nDone.")
print("Kept:", kept)
print("Skipped (no text):", skipped_text)
print("Skipped (audio read failed):", skipped_audio)
print("Metadata head:")
for l in lines[:3]:
    print("  ", l[:120])
PY

Loading dataset: Kostya165/syntetic_necoarc_rus
Indexing HF cache for audio files...
Indexed audio files: 0

Done.
Kept: 8600
Skipped (no text): 0
Skipped (audio read failed): 0
Metadata head:
   necoarc_000000|согласен. .. редко когда на дорогах встретишь|согласен. .. редко когда на дорогах встретишь
   necoarc_000001|этими вопросами должен заниматься следственный комитет|этими вопросами должен заниматься следственный ком
   necoarc_000002|ахуенно то как. не работайте, но платить не забывайте.|ахуенно то как. не работайте, но платить не забыва


Saving wavs: 100%|██████████| 8600/8600 [04:13<00:00, 33.92it/s]


In [ ]:
%%bash
set -e
source /content/py310/bin/activate

rm -rf /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2

echo "This will download XTTSv2. By running it you confirm you agree to CPML (non-commercial) or you have a commercial license."
echo "Answering 'y' to the prompt automatically..."

printf "y\n" | python -c "from TTS.utils.manage import ModelManager; ModelManager().download_model('tts_models/multilingual/multi-dataset/xtts_v2')"

echo "---- Model directory contents (top-level) ----"
ls -lah /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2 | sed -n '1,200p'

echo "---- Searching for needed files ----"
find /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2 -type f \( \
  -name 'model.pth' -o -name 'config.json' -o -name 'vocab.json' -o -name 'dvae.pth' -o -name 'mel_stats.pth' \
\) -print

This will download XTTSv2. By running it you confirm you agree to CPML (non-commercial) or you have a commercial license.
Answering 'y' to the prompt automatically...
 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]
 | | >  > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2
 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
---- Model directory contents (top-level) ----
total 1.8G
drwxr-xr-x 2 root root 4.0K Feb 26 17:55 .
drwxr-xr-x 3 root root 4.0K Feb 26 17:54 ..
-rw-r--r-- 1 root root 4.3K Feb 26 17:55 config.json
-rw-r--r-- 1 root root   32 Feb 26 17:55 hash.md5
-rw-r--r-- 1 root root 1.8G Feb 26 17:55 model.pth
-rw-r--r-- 1 root root 7.4M Feb 26 17:55 speakers_xtts.pth
-rw-r--r-- 1 root root   63 Feb 26 17:54 tos_agreed.txt
-rw-r--r-- 1 root root 353K Feb 

In [ ]:
%%bash
set -e
source /content/py310/bin/activate
python - <<'PY'
import os, statistics
import soundfile as sf

DATASET_DIR = "/content/xtts_dataset"
WAVS_DIR = os.path.join(DATASET_DIR, "wavs")
meta_path = os.path.join(DATASET_DIR, "metadata.csv")

with open(meta_path, "r", encoding="utf-8") as f:
    lines = [ln.strip() for ln in f if ln.strip()]

print("metadata lines:", len(lines))
wav_files = [fn for fn in os.listdir(WAVS_DIR) if fn.lower().endswith(".wav")]
print("wav files:", len(wav_files))

dur = []
for fn in wav_files[:2000]:
    p = os.path.join(WAVS_DIR, fn)
    info = sf.info(p)
    dur.append(info.frames / info.samplerate)

if dur:
    print("duration (sec) min/median/p95/max:",
          round(min(dur),2),
          round(statistics.median(dur),2),
          round(sorted(dur)[int(0.95*(len(dur)-1))],2),
          round(max(dur),2))
    print("NOTE: XTTS recipe defaults allow ~11.6s max wav. Longer clips may be skipped.")
PY


metadata lines: 8600
wav files: 8600
duration (sec) min/median/p95/max: 2.48 8.36 30.44 65.54
NOTE: XTTS recipe defaults allow ~11.6s max wav. Longer clips may be skipped.


In [ ]:
%%bash
set -e
cat > /content/train_xtts.py <<'PY'
import os
import torch

_original_load = torch.load
def patched_torch_load(*args, **kwargs):
    kwargs.setdefault("weights_only", False)
    return _original_load(*args, **kwargs)
torch.load = patched_torch_load

from trainer import Trainer, TrainerArgs
from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.layers.xtts.trainer.gpt_trainer import (
    GPTArgs,
    GPTTrainer,
    GPTTrainerConfig,
    XttsAudioConfig,
)

DATASET_DIR = "/content/xtts_dataset"
META_FILE   = os.path.join(DATASET_DIR, "metadata.csv")
LANGUAGE    = "ru"

RUN_NAME     = "XTTSv2_ru_finetune"
PROJECT_NAME = "XTTS_trainer"
OUT_PATH     = "/content/xtts_finetuned"

BATCH_SIZE       = 2
GRAD_ACCUM_STEPS = 32
NUM_WORKERS      = 2
LR = 5e-6

def find_model_dir():
    candidates = [
        os.path.join(os.path.expanduser("~"), ".local", "share", "tts",
                     "tts_models--multilingual--multi-dataset--xtts_v2"),
        os.path.join(os.path.expanduser("~"), ".cache", "tts",
                     "tts_models--multilingual--multi-dataset--xtts_v2"),
    ]
    for c in candidates:
        if os.path.isdir(c):
            return c
    return None

model_dir = find_model_dir()
if not model_dir:
    raise RuntimeError("XTTSv2 model not found. Run the download cell (Cell 3) first.")

required = ["model.pth", "config.json", "vocab.json", "dvae.pth", "mel_stats.pth"]
missing = [f for f in required if not os.path.isfile(os.path.join(model_dir, f))]
if missing:
    raise FileNotFoundError(f"Missing files in {model_dir}: {missing}")

with open(META_FILE, "r", encoding="utf-8") as f:
    first = None
    for line in f:
        line = line.strip()
        if not line:
            continue
        first = line.split("|", 1)[0]
        break
if not first:
    raise RuntimeError("metadata.csv is empty. Dataset creation failed.")

speaker_ref = os.path.join(DATASET_DIR, "wavs", f"{first}.wav")
if not os.path.isfile(speaker_ref):
    raise FileNotFoundError(f"Speaker reference wav not found: {speaker_ref}")

dataset_config = BaseDatasetConfig(
    formatter="ljspeech",
    dataset_name="custom",
    path=DATASET_DIR,
    meta_file_train=META_FILE,
    language=LANGUAGE,
)
DATASETS_CONFIG_LIST = [dataset_config]

model_args = GPTArgs(
    max_conditioning_length=132300,   # 6s @ 22.05k
    min_conditioning_length=66150,    # 3s @ 22.05k
    debug_loading_failures=False,
    max_wav_length=255995,            # ~11.6s @ 22.05k
    max_text_length=200,
    mel_norm_file=os.path.join(model_dir, "mel_stats.pth"),
    dvae_checkpoint=os.path.join(model_dir, "dvae.pth"),
    xtts_checkpoint=os.path.join(model_dir, "model.pth"),
    tokenizer_file=os.path.join(model_dir, "vocab.json"),
    gpt_num_audio_tokens=1026,
    gpt_start_audio_token=1024,
    gpt_stop_audio_token=1025,
    gpt_use_masking_gt_prompt_approach=True,
    gpt_use_perceiver_resampler=True,
)

audio_config = XttsAudioConfig(
    sample_rate=22050,
    dvae_sample_rate=22050,
    output_sample_rate=24000,
)

config = GPTTrainerConfig(
    output_path=OUT_PATH,
    model_args=model_args,
    run_name=RUN_NAME,
    project_name=PROJECT_NAME,
    dashboard_logger="tensorboard",
    logger_uri=None,
    audio=audio_config,
    batch_size=BATCH_SIZE,
    batch_group_size=48,
    eval_batch_size=BATCH_SIZE,
    num_loader_workers=NUM_WORKERS,
    eval_split_max_size=256,
    eval_split_size=0.1,
    print_step=50,
    plot_step=100,
    log_model_step=1000,
    save_step=2000,
    save_n_checkpoints=2,
    save_checkpoints=True,
    optimizer="AdamW",
    optimizer_wd_only_on_weights=True,
    optimizer_params={"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": 1e-2},
    lr=LR,
    lr_scheduler="MultiStepLR",
    lr_scheduler_params={"milestones": [50000 * 18, 150000 * 18, 300000 * 18], "gamma": 0.5, "last_epoch": -1},
    test_sentences=[
        {"text": "Привет! Это тестовое предложение для проверки голоса после дообучения.",
         "speaker_wav": [speaker_ref], "language": LANGUAGE},
        {"text": "Если всё работает, модель должна постепенно лучше воспроизводить тембр и произношение.",
         "speaker_wav": [speaker_ref], "language": LANGUAGE},
    ],
)

print("Loading train/eval samples...")
train_samples, eval_samples = load_tts_samples(
    DATASETS_CONFIG_LIST,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

print("Init model...")
model = GPTTrainer.init_from_config(config)

trainer = Trainer(
    TrainerArgs(
        restore_path=None,
        start_with_eval=True,
        grad_accum_steps=GRAD_ACCUM_STEPS,
        epochs=10,
    ),
    config,
    output_path=OUT_PATH,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)

print("Starting fine-tuning...")
trainer.fit()
PY

In [ ]:
%%bash
set -e
source /content/py310/bin/activate

python -m pip install -U huggingface_hub

python - <<'PY'
import os, shutil
from huggingface_hub import hf_hub_download

TARGET_DIR = "/root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2"
os.makedirs(TARGET_DIR, exist_ok=True)

for fn in ["dvae.pth", "mel_stats.pth"]:
    src = hf_hub_download(repo_id="coqui/XTTS-v2", filename=fn, revision="main")
    dst = os.path.join(TARGET_DIR, fn)
    shutil.copy2(src, dst)
    print("✅ copied", fn, "->", dst)

print("\nTop-level files now:")
print("\n".join(sorted(os.listdir(TARGET_DIR))[:50]))
PY

ls -lah /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2 | sed -n '1,120p'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.3/596.3 kB 9.1 MB/s  0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface_hub 0.36.2
    Uninstalling huggingface_hub-0.36.2:
      Successfully uninstalled huggingface_hub-0.36.2
✅ copied dvae.pth -> /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2/dvae.pth
✅ copied mel_stats.pth -> /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2/mel_stats.pth

Top-level files now:
config.json
dvae.pth
hash.md5
mel_stats.pth
model.pth
speakers_xtts.pth
tos_agreed.txt
vocab.json
total 2.0G
drwxr-xr-x 2 root root 4.0K Feb 26 18:02 .
drwxr-xr-x 3 root root 4.0K Feb 26 17:54 ..
-rw-r--r-- 1 root root 4.3K Feb 26 17:55 config.json
-rw-r--r-- 1 root root 201M Feb 26 18:02 dvae.pth
-rw-r--r-- 1 root root   32 Feb 26 17:55 hash.md5
-rw-r--r-- 1 root root 1.1K Feb 26 18:02 mel_stats.pth
-rw-r--r-- 1 root root 1.8G Feb 26 17:55 model.pth
-rw-r--r-- 1 root root 7.4M Feb 26 17:55 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.15.2 requires huggingface_hub<1.0,>=0.16.4, but you have huggingface-hub 1.5.0 which is incompatible.
transformers 4.37.2 requires huggingface-hub<1.0,>=0.19.3, but you have huggingface-hub 1.5.0 which is incompatible.


In [ ]:
%%bash
set -e
source /content/py310/bin/activate

python - <<'PY'
from pathlib import Path

p = Path("/content/train_xtts.py")
s = p.read_text().splitlines(True)

out = []
inside_trainerargs = False
paren_balance = 0

for line in s:
    if "TrainerArgs(" in line:
        inside_trainerargs = True
        paren_balance = line.count("(") - line.count(")")
        out.append(line)
        continue

    if inside_trainerargs:
        paren_balance += line.count("(") - line.count(")")
        if line.strip().startswith("epochs="):
            continue
        if "restore_path=None" in line:
            line = line.replace("restore_path=None", 'restore_path=""')
        out.append(line)
        if paren_balance <= 0:
            inside_trainerargs = False
        continue

    out.append(line)

txt = "".join(out)

marker = '\nprint("Loading train/eval samples...")'
if "config.epochs" not in txt:
    txt = txt.replace(marker, "\nconfig.epochs = 10\n" + marker)

p.write_text(txt)
print("✅ Patched /content/train_xtts.py (moved epochs -> config.epochs)")
PY

python -u /content/train_xtts.py